# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

Classification, because we are trying to label students into two classes (passed_yes, passed_no) based upon a training set of historical students and their graduation outcomes. A similar regression problem could be to predict their final grade (assuming a continues 0-100 score), as opposed to just their final status.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [6]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [7]:
# TODO: Calculate number of students
n_students = student_data.count()[0]


# TODO: Calculate number of features
n_features = student_data.count(1)[0]

# TODO: Calculate passing students
n_passed = student_data[student_data['passed'] == 'yes'].count()[0]

# TODO: Calculate failing students
n_failed = student_data[student_data['passed'] == 'no'].count()[0]

# TODO: Calculate graduation rate
grad_rate = float(n_passed) / n_students * 100

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 31
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [8]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [9]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [11]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split


# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=1)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

1) Gaussian Naive Bayes: Naive Bayes models can be used in a wide variety of classification problems.  They have been successfully applied to [email spam filtering](https://en.wikipedia.org/wiki/Naive_Bayes_spam_filtering) and [automated medical diagnostics](http://scialert.net/fulltext/?doi=itj.2012.1166.1174&org=11).  The strengths of Naive Bayes are that it can fit relatively small data sets and that its computational complexity scales linearly (so it can handle very large data sets without enormous computational times).  One weakness of NB is that it doesn't model interactions between features (non-linear relationships).  Another weakness of the model is that it can be skewed by scarce data points; for example, if a NB model encounters a testing observation in which one of its features presents a new value that was not encountered during training, then the model may assign a 0% probability to that outcome.  This model is potentially a good candidate for our problem because it can be trained and tested quickly and can identify linear relationships within the data. 

2) Random Forest: Random Forest Decision Trees offer the simplicity of Decision Trees with safeguards against over-fitting through the use of ensembling shallower trees.  Random Forest has been used for many types of classification problems, including the classic [Titanic survival](https://www.kaggle.com/c/titanic) and [click stream analysis](https://arxiv.org/pdf/1511.06247.pdf).  The strengths of the Random Forest algorithm is that it can easily capture interactions between features and resists over-fitting through ensembling.  A main weakness of Random Forest is its relative opacity, particularly in comparison to a singular Decision Tree.  Random Forest could be a good candidate for our problem because of its ability to extract relationships between the features to contstruct more complex decision formulas.  However, our training set is relatively small, so Random Forest may not be able to leverage its ensembling quality as effectively.

3) SVM: Support Vector Classifers (SVC's) can be used to create highly complex and nonlinear decision boundaries, making them good candidates for high dimensionality classification problems, particularly those that are not linearly seperable.  SVC's have been used in [road detection](http://web.mit.edu/mobility/publications/Iagnemma_IVSS_10.pdf) problems for autonomous driving, [detection of microcalcifications](https://www.cse.unr.edu/~looney/cs773b/SVMmicrocalcification.pdf) as indicators of tumors in mammograms, and to predict [long-run changes in the price of technology stocks](https://www.cs.princeton.edu/sites/default/files/uploads/saahil_madge.pdf).  The strenghts of the SVM is its ability to construct and apply complex, non-linear decision boundaries, giving it the ability to accurately learn complex relationships betweens features and target.  The weakness of the SVM is primarily its computational complexity, sometimes making it ill-suited for training on large data sets.  Additionally, if not properly cross-validated, the SVM's flexibility can lead to over-fitting the training set.  In this instance, the SVC is a good candidate for our problem because we have a relatively small data set with a potentially complex relationship between the features (student characteristics) and the target (passing status).  

4) GradientBoostingClassifier: Similar to Random Forest, Gradient Boosting Classifiers (GBC's) utilizes ensembling techniques to improve a group of weak classifiers into a single strong classifier.  GBC's have been used in many traditional classification problems.  The strength of GBC are similar to Random Forest, but can potentially outperform by iteratively generating its weak classifiers, rather than generating them randomly.  GBC's perform well in many classification settings, but can be subject to over-fitting with smaller data sets.  GBC is a good candidate for our problem for similar reasons to the Random Forest (extracting feature relationships, relatively inexpensive training cost); however, without parameter tuning there is a risk that even the weak classifiers risk over-fitting our smaller training set.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [12]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [13]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

# TODO: Initialize the three models
clf_A = GaussianNB()
clf_B = RandomForestClassifier(random_state=1)
clf_C = SVC(random_state=1)
clf_D = GradientBoostingClassifier(random_state=1)

# TODO: Set up the training set sizes
X_train_100 = X_train.sample(n=100)
y_train_100 = y_train.ix[X_train_100.index]


X_train_200 = X_train.sample(n=200)
y_train_200 = y_train.ix[X_train_200.index]

X_train_300 = X_train
y_train_300 = y_train

X_train_10000 = X_train.sample(n=10000, replace=True)
y_train_10000 = y_train.ix[X_train_10000.index]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
sets = [(X_train_100, y_train_100), (X_train_200, y_train_200), (X_train_300, y_train_300), (X_train_10000, y_train_10000)]
clfs = [clf_A, clf_B, clf_C, clf_D]

for clf in clfs:
    for (X_train, y_train) in sets:
        train_predict(clf, X_train, y_train, X_test, y_test)

Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0017 seconds
Made predictions in 0.0008 seconds.
F1 score for training set: 0.6847.
Made predictions in 0.0008 seconds.
F1 score for test set: 0.4348.
Training a GaussianNB using a training set size of 200. . .
Trained model in 0.0019 seconds
Made predictions in 0.0005 seconds.
F1 score for training set: 0.7881.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.7132.
Training a GaussianNB using a training set size of 300. . .
Trained model in 0.0010 seconds
Made predictions in 0.0005 seconds.
F1 score for training set: 0.7921.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.6720.
Training a GaussianNB using a training set size of 10000. . .
Trained model in 0.0270 seconds
Made predictions in 0.0106 seconds.
F1 score for training set: 0.7874.
Made predictions in 0.0004 seconds.
F1 score for test set: 0.6880.
Training a RandomForestClassifier using a training set size of 100. . .
Traine

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - GaussianNB**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0021           |      0.0006             |      0.4545      |     0.3117      |
| 200               |        0.0013           |        0.0007          |        0.8315    |      0.7520     |
| 300               |         0.0021          |     0.0006             |    0.6610        |    0.5524       |

** Classifer 2 - RandomForestClassifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0576           |     0.0013             |       0.9925     |          0.7344 |
| 200               |     0.0600              |     0.0013             |      0.9962      |         0.7231  |
| 300               |          0.0258         |     0.0015             |       0.9901     |     0.6963     |

** Classifer 3 - SVC**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |       0.0113            |    0.0010              |        0.8874    |       0.8000    |
| 200               |       0.0046            |    0.0026              |        0.8599    |       0.7917    |
| 300               |       0.0106            |    0.0029              |        0.8120    |       0.7898    |
| 10,000            |          2.1810         |     0.0133             |       0.9779     |     0.7077     |

** Classifer 4 - GradientBoostingClassifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |       0.0714            |    0.0006              |        1.0000    |       0.7786    |
| 200               |       0.1012            |    0.0007              |        1.0000    |       0.8000    |
| 300               |       0.1021            |    0.0006              |        0.9621    |       0.7692    |
| 10,000            |          2.1850         |    0.0006              |       1.000     |     0.6825     |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

Based on the experiments, I believe the most appropriate model for predicting graduation status amongst the student_data set is the Support Vector Machine Classifier (SVC).  

Given the size of the historical data set, training and prediction times were not significant factors in determining the ideal model.  Training times ranged from ~3 hundredths of a second to ~1 tenth of second.  However, we would note that computational complexity for SVM's can be very steep.  As an example of this, we ran a fourth simulation on the SVC with 10,000 data points (sampled from X_train with replacement) and found that the training time increased nearly 200-fold to ~2.2 seconds.  While 2.2 seconds is still a very reasonable training time, if we expected that the historical data set could be expanded signficantly - for example, if we gained access to a national data base - then the non-linear increase in training training times for SVC could become a stronger consideration.

The primary motivation in support of the SVC model is its F1 score performance amongst both training and test data.  On the test data, the SVC - without parameter tuning - scored around 0.79 to 0.80, consistently.  Notably, this performance was superior to both the GaussianNB and RandomForestClassifier models, and roughly equivalent to the performance of the GradientBoostingClassifier (GBC). However, we note that the relatively large dispartity between the GBC's performance on the training set versus the testing set indicate a strong likelihood that the model is overfitting the training data and suffering from high variance.  If we had or expected access to more training data, then we could potentially combat this overfitting by increasing the training set size; however, given the small training set that we have available, it is most likely that our parameter tuning for the GBC would increase the bias in the model, which would likely bring down training performance and may or may not improve testing performance.  In the case of the SVC, however, there is relatively less drop-off between training and testing performance, which gives us greater confidence in its generalizability.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

The predictive model looks at characteristics of the student, including age, gender, family educational and employment background, health, and recreational preferences, in order to make a prediction about that student's graduation status (pass_yes or pass_no).  The intention of the model is to provide teachers or counselors with greater predictive insights into which of their students are most likely to fail, with the hope that identifying high-risk students early will provide better targeting for interventional services.  When a new student is scored through the model, their background information is provided to the model and it will return a prediction of whether that student will pass or fail.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [14]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import make_scorer

# TODO: Create the parameters list you wish to tune
parameters = [
    {'C': [1, 10, 100], 'gamma': [ 1.0/32, 0.1, 0.001], 'kernel': ['rbf']},
    #{'C': [1, 10, 100], 'kernel': ['linear']},
    ]

# TODO: Initialize the classifier
clf = SVC(random_state=1)
#clf = GradientBoostingClassifier(random_state=1)
print clf.get_params().keys()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(lambda X, y: f1_score(X, y, pos_label='yes'))

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_
print grid_obj.best_params_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

['kernel', 'C', 'verbose', 'probability', 'degree', 'shrinking', 'max_iter', 'decision_function_shape', 'random_state', 'tol', 'cache_size', 'coef0', 'gamma', 'class_weight']
{'kernel': 'rbf', 'C': 1, 'gamma': 0.1}
Made predictions in 0.3241 seconds.
Tuned model has a training F1 score of 1.0000.
Made predictions in 0.0031 seconds.
Tuned model has a testing F1 score of 0.8344.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

The final model's F1 scores were 1.0000 and 0.8344 for training and testing, respectively.  Both of those values are an increase from the untuned model; however, the perfect training score and widened drop-off between the training and testing scores indicate that the tuned model may be overfitting.  

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.